In [1]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from EnergySaverLLM.Agent import ChargingAgent, _insert_params, _replace_json
from flaml.autogen.code_utils import extract_code

In [2]:
print(extract_code("""
```JSON
"end_charge_time": 9,
"end_charge": .95,
```
"""))

[('JSON', '"end_charge_time": 9,\n"end_charge": .95,')]


In [3]:
# global params_filepath
params_filepath = "EnergySaverLLM/Model/params/EVCharging.json"

In [4]:
with open(params_filepath) as f:
    json_str = f.read()

In [5]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        #"model": ["gpt-4", "gpt-3.5-turbo"],
        "model": ["gpt-3.5-turbo"],
    },
)

In [6]:
autogen.oai.ChatCompletion.start_logging()

In [7]:
code_path = "EnergySaverLLM/Model/EVCharging.py"

In [8]:
with open(code_path) as f:
    code = f.read()

In [9]:
# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from datetime import datetime
from gurobipy import GRB, Model
import json

# Fetching data from JSON file

params_filepath = "EnergySaverLLM/Model/params/EVCharging.json"

stored_params = json.load(open(params_filepath))

.
.
.

    values = model.getAttr("X", all_vars)
    names = model.getAttr("VarName", all_vars)

    for i, time_str in enumerate(keys):
        if(values[i]>0):
            print(f"Scheduled consumption at Hour {time_str} : {int(values[i]):d} KWH")

else:
    print("Not solved to optimality. Optimization status:", model.status)



In [10]:
example_qa = """
----------
Instruction: Charge the car till 9 AM.
Answer Code:
```JSON
"end_charge_time": 9
```

----------
Question: Charge the car to full charge by 9 AM
Answer Code:
```JSON
"end_charge": 1.00,
"end_charge_time": 9
```
"""

In [11]:
%%capture
agent = ChargingAgent(
    name="Tesla Charging Example",
    source_code=code,
    debug_times=1,
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [12]:
user.initiate_chat(agent, message="Complete charging battery to 100% by 9 AM tomorrow")

user (to Tesla Charging Example):

Complete charging battery to 100% by 9 AM tomorrow

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

"end_charge": 1.0,
"end_charge_time": 9

--------------------------------------------------------------------------------


NameError: name 'src_code' is not defined

In [14]:
user.initiate_chat(agent, message="Complete charging battery to 100% by 9 AM tomorrow")

user (to Tesla Charging Example):

Complete charging battery to 100% by 9 AM tomorrow

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
end_charge_level = 1
end_time_H = 9
this_charge_KWH = (end_charge_level - start_charge_level) * total_battery_capacity
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
end_charge_level = 1
end_time_H = 9
this_charge_KWH = (end_charge_level - start_charge_level) * total_battery_capacity
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)

--- One-Word Answer: SAFE or DANGER ---


----------------------------------------------------------------

In [17]:
user.initiate_chat(agent, message="Reduce max allowed electricity cost to 15 dollars")

user (to Tesla Charging Example):

Reduce max allowed electricity cost to 15 dollars

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
max_elec_cost = 15
model.addConstr(sum(x[i] * elec_costs[i] for i in keys) <= max_elec_cost)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
max_elec_cost = 15
model.addConstr(sum(x[i] * elec_costs[i] for i in keys) <= max_elec_cost)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Tesla Charging Example):

SAFE

--------------------------------------------------------------------------------
{'0': 0.3, '1': 0.3, '2': 0.3, '3': 0.3, '4': 0.3, '5': 0.3, '6': 0.3, '7':

In [18]:
user.initiate_chat(agent, message="Charge the car to full charge by 6 AM tomorrow")

user (to Tesla Charging Example):

Charge the car to full charge by 6 AM tomorrow

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
end_charge_level = 1
this_charge_KWH = (end_charge_level-start_charge_level)*total_battery_capacity
model.addConstr(sum(x[i] for i in keys) == this_charge_KWH)
end_time_H = 30
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
end_charge_level = 1
this_charge_KWH = (end_charge_level-start_charge_level)*total_battery_capacity
model.addConstr(sum(x[i] for i in keys) == this_charge_KWH)
end_time_H = 30
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_

In [20]:
user.initiate_chat(agent, message="Charge the car by 9 AM tomorrow")

user (to Tesla Charging Example):

Charge the car by 9 AM tomorrow

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
end_time_H = 33
upper_bounds = [0]*len(elec_costs)
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
end_time_H = 33
upper_bounds = [0]*len(elec_costs)
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Tesla Charging Example):

SAFE

--------------------------------------------------------------------------------
{'0': 0.3, 